In [1]:
import turtle
import math

In [2]:
class Matrix(object):
    def __init__(self, h, w = 1):
        if type(h) == int:
            self.dat = [[0]*w for _ in range(h)]
        elif type(h) == list and (type(h[0]) == float or type(h[0]) == int):
            self.dat = [[p] for p in h]
        else:
            self.dat = h            
        
    def __repr__(self):
        return 'Matrix(%s)'%self.dat
    
    def __mul__(self, r):
        res = Matrix(len(self.dat), len(r.dat[0]))
        for p in range(len(self.dat)):
            for q in range(len(r.dat[0])):
                val = 0
                for ind in range(len(self.dat[0])):
                    val += self.dat[p][ind] * r.dat[ind][q]
                res.dat[p][q] = val
        return res
    
    def transpose(self):
        h, w = len(self.dat), len(self.dat[0])
        tm = Matrix(w, h)
        for p in range(h):
            for q in range(w):
                tm.dat[q][p] = self.dat[p][q]
        return tm
    
    def height(self):
        return len(self.dat)
    
    def width(self):
        return len(self.dat[0])
    
    def set(self, p, q, val):
        self.dat[p][q] = val
    
    def get(self, p, q):
        return self.dat[p][q]
    
    def copy(self):
        return [t.copy() for t in self.dat]
    
    def first_column_to_list(self):
        h = len(self.dat)
        ret = [0] * h
        for p in range(h):
            ret[p] = self.dat[p][0]
        return ret
        

In [3]:
def create_mnk_matrix(dat):
    max_order = 3
    order = min(max_order, len(dat)-1)
    dat = Matrix(dat)
    b = Matrix(dat.height(), order + 1)
    y = Matrix(dat.height(), 1)
    for p in range(dat.height()):
        for q in range(order+1):
            b.set(p, q, dat.get(p, 0)**q)
        y.set(p, 0, dat.get(p, 1))
    return b, y

In [4]:
def gauss_solve(a, y):
    n = a.width()
    x = Matrix(n)
    for pos in range(n-1):
        maxel_ind = pos
        maxel = math.fabs(a.get(pos,pos))
        for ind in range(pos+1, n):
            if maxel < math.fabs(a.get(ind, pos)):
                maxel = math.fabs(a.get(ind, pos))
                maxel_ind = ind
        if maxel_ind != pos:
            for temp_ind in range(pos, n):
                temp_val = a.get(ind, temp_ind)
                a.set(ind, temp_ind, a.get(pos, temp_ind))
                a.set(pos, temp_ind, temp_val)
            temp_val = y.get(pos, 0)
            y.set(pos, 0, y.get(maxel_ind, 0))
            y.set(maxel_ind, 0, temp_val)
        for victim in range(pos+1, n):
            magic = a.get(victim, pos)/a.get(pos, pos)
            y.set(victim, 0, y.get(victim, 0) - y.get(pos, 0)*magic)
            for subvictim in range(pos, n):
                a.set(victim, subvictim, a.get(victim, subvictim) - a.get(pos, subvictim)*magic)
    x.set(n-1, 0, y.get(n-1, 0)/a.get(n-1, n-1))
    for pos in range(n-2, -1, -1):
        right = y.get(pos, 0)
        for p in range(pos+1, n):
            right -= x.get(p, 0)*a.get(pos, p)
        x.set(pos, 0, right/a.get(pos, pos))
    return x

In [5]:
def draw_points(a):
    turtle.up()
    turtle.clear()

    print('a=',a)
    b, y = create_mnk_matrix(a)
    print('b=', b, 'y=', y)

    ys = b.transpose() * y
    bs = b.transpose() * b
    poly = gauss_solve(bs, ys).first_column_to_list()
    print('poly = ', poly)
    print()
    
    for point in a:
        turtle.pencolor((0, 0, 0))
        real_point = tuple(t*100 for t in point)
        print('real_point = ', real_point)
        y = 0
        x=point[0]
        for ind, coef in enumerate(poly):
            y += coef * x ** ind
        approx_point = (x*100, y*100)
        print('approx_point = ', approx_point)
        turtle.goto(*real_point)
        turtle.dot()

    minx = min(t[0] for t in a) - 1
    maxx = max(t[0] for t in a) + 1
    d = maxx - minx

    turtle.color((0, 1, 0))
    for x in range(int(100*(minx - d/2)), int(100*(maxx + d/2))):
        y = 0
        for ind, coef in enumerate(poly):
            y += coef * (x/100) ** ind
        approx_point = (x, y*100)
        turtle.goto(*approx_point)
        turtle.down()
    print()
    print()

In [6]:
class MyPoints(object):
    def __init__(self):
        self.dat = []
    def my_click(self,x,y):
        self.dat.append((x/100,y/100))
        if len(self.dat) > 1:
            draw_points(self.dat)

In [7]:
points = MyPoints()

turtle.forward(10)
turtle.getscreen().onclick(points.my_click)
turtle.tracer(0)
turtle.mainloop()

a= [(-5.85, 0.87), (-3.08, -0.19)]
b= Matrix([[1.0, -5.85], [1.0, -3.08]]) y= Matrix([[0.87], [-0.19]])
poly =  [-1.3686281588447635, -0.3826714801444039]

real_point =  (-585.0, 87.0)
approx_point =  (-585.0, 86.9999999999999)
real_point =  (-308.0, -19.0)
approx_point =  (-308.0, -18.99999999999995)


a= [(-5.85, 0.87), (-3.08, -0.19), (1.64, 0.8)]
b= Matrix([[1.0, -5.85, 34.2225], [1.0, -3.08, 9.4864], [1.0, 1.64, 2.6895999999999995]]) y= Matrix([[0.87], [-0.19], [0.8]])
poly =  [0.056495189858071906, 0.32364173463616724, 0.07909442494743245]

real_point =  (-585.0, 87.0)
approx_point =  (-585.0, 87.00000000000006)
real_point =  (-308.0, -19.0)
approx_point =  (-308.0, -19.000000000000007)
real_point =  (164.0, 80.0)
approx_point =  (164.0, 80.00000000000004)


a= [(-5.85, 0.87), (-3.08, -0.19), (1.64, 0.8), (5.56, 1.71)]
b= Matrix([[1.0, -5.85, 34.2225, -200.20162499999995], [1.0, -3.08, 9.4864, -29.218112], [1.0, 1.64, 2.6895999999999995, 4.410943999999999], [1.0, 5.56, 30.9135999